In [1]:
sc.master

'local[*]'

## 1  准备数据
StumbleUpon Evergreen数据，来源于Kaggle中的一个题目StumbleUpon Evergreen Classification Challenge。

StumbleUpon 是一个个性化推荐引擎，根据用户的兴趣行为给用户推荐网页，而有些网页内容是即时性（ephemeral）的，比如新闻股票网页（用户短暂感兴趣），有些网页是长久性的（evergreen）如体育，理财等（用户持续感兴趣）。现要分辨网页是ephemeral的还是evergreen的，以便向用户推荐更加准确的网页。

这是一个二分类问题。

查看StumbleUpon数据的详细信息：https:www.kaggle.com/c/stumbleupon/data

下载StumbleUpon数据，train.tsv和test.tsv。

分析train.tsv的字段：

- 0~2列忽略，url，urlid对网站是否evergreen关系不大
- 第3列，categorical features分类特征，网页分类如：business，health, sports……
- 第4~25列，numerical features数值特征，有关此网页的数值特征，例如链接的数目，图片的比例等
- 第26列，label分类标签，1表示evergreen（用户长久感兴趣的网页），0表示non-evergreen（用户短暂感兴趣的网页）

test.tsv没有第26列的label分类标签，其余和test.tsv相同


In [17]:
## 读取train.tsv
print("开始导入数据...")
rawDataWithHeader = sc.textFile("data/train.tsv")
## 取出前2项数据
rawDataWithHeader.take(2)

开始导入数据...


['"url"\t"urlid"\t"boilerplate"\t"alchemy_category"\t"alchemy_category_score"\t"avglinksize"\t"commonlinkratio_1"\t"commonlinkratio_2"\t"commonlinkratio_3"\t"commonlinkratio_4"\t"compression_ratio"\t"embed_ratio"\t"framebased"\t"frameTagRatio"\t"hasDomainLink"\t"html_ratio"\t"image_ratio"\t"is_news"\t"lengthyLinkDomain"\t"linkwordscore"\t"news_front_page"\t"non_markup_alphanum_characters"\t"numberOfLinks"\t"numwords_in_url"\t"parametrizedLinkRatio"\t"spelling_errors_ratio"\t"label"',
 '"http://www.bloomberg.com/news/2010-12-23/ibm-predicts-holographic-calls-air-breathing-batteries-by-2015.html"\t"4042"\t"{""title"":""IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries"",""body"":""A sign stands outside the International Business Machines Corp IBM Almaden Research Center campus in San Jose California Photographer Tony Avelar Bloomberg Buildings stand at the International Business Machines Corp IBM Almaden Research Center campus in the S

从运行结果看出：第一项是字段名(特征名称)不是数据（要进行删除）；每一项数据以制表符”\t”分隔；每个字段前后都有双引号“””分隔。除此之外，在train.tsv表中有些字段有缺失值用”?”表示。

针对以上问题对train.tsv进行处理

In [4]:
## 读取train.tsv
print("开始导入数据...")
rawDataWithHeader = sc.textFile("data/train.tsv")
## 取第一项数据
header = rawDataWithHeader.first()
## 剔除字段名（特征名）行，取数据行
rawData = rawDataWithHeader.filter(lambda x:x!=header)
## 将双引号"替换为空字符（剔除双引号）
rData = rawData.map(lambda x:x.replace("\"",""))
## 以制表符分割每一行
lines = rData.map(lambda x: x.split("\t"))
print("共有："+str(lines.count())+"项数据")

开始导入数据...
共有：7395项数据


In [16]:
# 0~2列忽略，url，urlid对网站是否evergreen关系不大
lines.first()[3:]

['business',
 '0.789131',
 '2.055555556',
 '0.676470588',
 '0.205882353',
 '0.047058824',
 '0.023529412',
 '0.443783175',
 '0',
 '0',
 '0.09077381',
 '0',
 '0.245831182',
 '0.003883495',
 '1',
 '1',
 '24',
 '0',
 '5424',
 '170',
 '8',
 '0.152941176',
 '0.079129575',
 '0']

## 2  处理特征

该数据集的第3个字段是alchemy_category网页分类，是一个离散值特征，要采用OneHotEncode的方式进行编码转换为数值特征，主要过程如下：

1. 创建categoriesMap字典，key为网页类别名，value为数字（网页类别名的索引值），每个类别名对应一个索引值
2. 根据categoriesMap字典查询每个alchemy_category特征值对应的索引值，例如business的索引值categoryIdx为2
3. 根据categoryIdx=2，以OneHotEncodeer的方式转换为一个列表categoryFeatures List，该列表长度为14（统计所有网页类别），categoryIdx=2对应的列表为[0,0,1,0,0,0,0,0,0,0,0,0,0,0]。

建立categoriesMap网页分类字典

In [12]:
categoriesMap = lines.map(lambda x: x[3]).distinct().zipWithIndex().collectAsMap()

其中，lines.map()表示处理之前读取的数据的每一行，.map(lambda fields: fileds[3])表示读取第3个字段，.distinct()保留不重复数据，.zipWithIndex()将第3个字段中不重复的数据进行编号，.collectAsMap()转换为dict字典格式


In [13]:
categoriesMap

{'business': 0,
 'recreation': 1,
 'health': 2,
 'sports': 3,
 '?': 4,
 'arts_entertainment': 5,
 'science_technology': 6,
 'gaming': 7,
 'culture_politics': 8,
 'computer_internet': 9,
 'law_crime': 10,
 'religion': 11,
 'weather': 12,
 'unknown': 13}

将每个alchemy_category网页分类特征值转化为列表categoryFeatures List

In [18]:
import numpy as np
categoryIndex = categoriesMap[lines.first()[3]]
OneHot = np.zeros(len(categoriesMap))
OneHot[categoryIndex]=1
print(OneHot)

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


对于第4~25字段的数值特征，要转换为数值，用float函数将字符串转换为数值，同时简单处理缺失值”?”为0.

整个处理特征的过程可以封装成一个函数：

In [19]:
import numpy as np

def convert(v):
    """处理数值特征的转换函数"""
    return (0 if v=="?" else float(v))

def process_features(line, categoriesMap, featureEnd):
    """处理特征，line为字段行，categoriesMap为网页分类字典，featureEnd为特征结束位置，此例为25"""
    ## 处理alchemy_category网页分类特征
    categoryIdx = categoriesMap[line[3]]
    OneHot = np.zeros(len(categoriesMap))
    OneHot[categoryIdx] = 1
    ## 处理数值特征
    numericalFeatures = [convert(value) for value in line[4:featureEnd]]
    # 返回拼接的总特征列表
    return np.concatenate((OneHot, numericalFeatures))

## 3、处理label分类标签

定义一个函数：

In [20]:
def process_label(line):
    return float(line[-1])  # 最后一个字段为类别标签

## 4、构建模型所需数据格式

Spark Mllib分类任务支持的数据类型为LabeledPoint格式，LabeledPoint数据由标签label和特征feature组成。构建LabeledPoint数据：

In [22]:
from pyspark.mllib.regression import LabeledPoint
labelpointRDD = lines.map(lambda r: LabeledPoint(process_label(r), process_features(r,categoriesMap, len(r)-1)))

In [23]:
labelpointRDD.take(1)

[LabeledPoint(0.0, [1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.789131,2.055555556,0.676470588,0.205882353,0.047058824,0.023529412,0.443783175,0.0,0.0,0.09077381,0.0,0.245831182,0.003883495,1.0,1.0,24.0,0.0,5424.0,170.0,8.0,0.152941176,0.079129575])]

## 5、划分训练集、验证集以及测试集

按照7:1:2的比例划分训练集、验证集以及测试集

In [24]:
## 划分训练集、验证集和测试集
(trainData, validationData, testData) = labelpointRDD.randomSplit([7,1,2])
print("训练集样本个数："+str(trainData.count()) + "验证集样本个数："+str(validationData.count())+ "测试集样本个数："+str(testData.count()))

训练集样本个数：5137验证集样本个数：791测试集样本个数：1467


In [25]:
# 将数据暂存到内存中，加快后续运算效率
trainData.persist()
validationData.persist()
testData.persist()

PythonRDD[42] at RDD at PythonRDD.scala:53

## 6 训练模型
选择Spark MLlib中的决策树DecisionTree模块中的trainClassifier方法进行训练并建立模型：

DecisionTree.trainClassifier(input, numClasses, categoricalFeaturesInfo, impurity,maxDepth,maxBins)
参数说明如下：

- (1) input：输入的训练数据，数据格式为LabeledPoint数据
- (2) numClasses：指定分类数目
- (3) categoricalFeaturesInfo：设置分类特征字段信息，本例采用OneHot编码处理分类特征字段，故这里设置为空字典dict()
- (4) impurity：决策树的impurity评估方法（划分的度量选择）：gini基尼系数，entropy熵
- (5) maxDepth：决策树最大深度
- (6) maxBins：决策树每个节点的最大分支数


In [26]:
from pyspark.mllib.tree import DecisionTree
model = DecisionTree.trainClassifier(trainData, numClasses=2,categoricalFeaturesInfo={}, impurity="entropy", maxDepth=5,maxBins=5)

## 7   评估模型
使用AUC(Area under the Curve of ROC)来对模型进行评估，接收者操作特征(Receiver Operating Characteristic , ROC)曲线是一种比较分类器模型有用的可视化工具。

ROC曲线显示了给定模型的真正例率(TPR=TP/P)(纵轴)和假正例率(FPR=FP/N)(横轴)之间的权衡。TPR的增加以FPR的增加为代价。ROC曲线下方的面积是模型准确率的度量：AUC

- AUC=1：预测准确率100%
- 0.5 < AUC <1：优于随机猜测，具有预测意义
- AUC=0.5: 与随机猜测一样，没有预测意义
- AUC<0.5: 比随机预测还差

Spark Mllib提供了BinaryClassificationMetrics计算AUC的方法。

首先创建predict_real列表，列表的每个元素为一个元组(predict,real)，其中predict为预测结果，real为实际标签


In [27]:
## 创建predict_real列表
predict = model.predict(validationData.map(lambda p:p.features))
predict_real = predict.zip(validationData.map(lambda p: p.label))
predict_real.take(5)

[(1.0, 0.0), (1.0, 1.0), (0.0, 0.0), (0.0, 1.0), (1.0, 0.0)]

接着使用BinaryClassificationMetrics计算AUC

In [28]:
## 使用BinaryClassificationMetrics计算AUC
from pyspark.mllib.evaluation import BinaryClassificationMetrics
metrics = BinaryClassificationMetrics(predict_real)
print("AUC="+str(metrics.areaUnderROC))

AUC=0.6324127403568633


## 8  模型参数选择
DecisionTree的参数impurity,maxDepth,maxBins会影响模型的准确率及训练的时间，下面对不同模型参数取值进行测试评估。

创建trainEvaluateModel函数包含训练与评估功能，并计算训练评估的时间。

In [52]:
## 创建trainEvaluateModel函数包含训练与评估功能，并计算训练评估的时间。
from time import time

def trainEvaluateModel(trainData, validationData, impurityParm, maxDepthParm, maxBinsParm):
    startTime = time()
    ## 创建并训练模型
    model = DecisionTree.trainClassifier(trainData, numClasses=2,categoricalFeaturesInfo={}, 
                                         impurity=impurityParm, maxDepth=maxDepthParm,maxBins=maxBinsParm)
    ## 计算AUC
    predict = model.predict(validationData.map(lambda p:p.features))
    predict_real = predict.zip(validationData.map(lambda p: p.label))
    metrics = BinaryClassificationMetrics(predict_real)
    AUC = metrics.areaUnderROC
    duration = time() - startTime   # 持续时间
#     print("训练评估：参数"+"impurity="+str(impurityParm) + 
#          "   maxDepth="+str(maxDepthParm)+"   maxBins="+str(maxBinsParm)+"\n"+
#          "===>消耗时间="+str(duration)+"   结果AUC="+str(AUC))
    return AUC, duration, impurityParm, maxDepthParm, maxBinsParm, model

### 1、评估impurity参数

In [41]:
## 评估impurity参数
impurityList=["gini","entropy"]
maxDepthList = [10]
maxBinsList = [10]

## 返回结果存放至metries中
metrics = [trainEvaluateModel(trainData, validationData, impurity, maxDepth, maxBins)
          for impurity in impurityList
          for maxDepth in maxDepthList
          for maxBins in maxBinsList]

训练评估：参数impurity=gini   maxDepth=10   maxBins=10
===>消耗时间=1.999284267425537   结果AUC=0.6431558423233321
训练评估：参数impurity=entropy   maxDepth=10   maxBins=10
===>消耗时间=1.731438159942627   结果AUC=0.6598188745077415


In [42]:
# 查看metrics
print(metrics)

[(0.6431558423233321, 1.999284267425537, 'gini', 10, 10, DecisionTreeModel classifier of depth 10 with 557 nodes), (0.6598188745077415, 1.731438159942627, 'entropy', 10, 10, DecisionTreeModel classifier of depth 10 with 471 nodes)]


以图表形式显示训练评估结果

In [44]:
import pandas as pd
IndexList = impurityList
df= pd.DataFrame(metrics,index=IndexList,columns=["AUC","duration","impurity","maxDepth","maxBins","model"])
df

,AUC,duration,impurity,maxDepth,maxBins,model
gini,0.643156,1.999284,gini,10,10,DecisionTreeModel classifier of depth 10 with ...
entropy,0.659819,1.731438,entropy,10,10,DecisionTreeModel classifier of depth 10 with ...


### 2、评估maxDepth参数

In [46]:
## 评估maxDepthList参数
impurityList = ["gini"]
maxDepthList = [3,5,10,15,20,25]
maxBinsList = [10]
#
# 存放结果存放至metrics中
metrics = [trainEvaluateModel(trainData, validationData, impurity, maxDepth, maxBins)
          for impurity in impurityList
          for maxDepth in maxDepthList
          for maxBins in maxBinsList]

训练评估：参数impurity=gini   maxDepth=3   maxBins=10
===>消耗时间=1.832421064376831   结果AUC=0.6098233641622945
训练评估：参数impurity=gini   maxDepth=5   maxBins=10
===>消耗时间=1.8724696636199951   结果AUC=0.6529112202881076
训练评估：参数impurity=gini   maxDepth=10   maxBins=10
===>消耗时间=1.16658353805542   结果AUC=0.6431558423233321
训练评估：参数impurity=gini   maxDepth=15   maxBins=10
===>消耗时间=1.6244165897369385   结果AUC=0.6239144656669703
训练评估：参数impurity=gini   maxDepth=20   maxBins=10
===>消耗时间=1.8939316272735596   结果AUC=0.6245333966160832
训练评估：参数impurity=gini   maxDepth=25   maxBins=10
===>消耗时间=2.4340503215789795   结果AUC=0.6245333966160832


### 3、评估maxBins参数

In [48]:
## 评估maxBins参数
impurityList = ["gini"]
maxDepthList = [10]
maxBinsList = [3,5,10,15,100,200]
#
# 存放结果存放至metrics中
metrics = [trainEvaluateModel(trainData, validationData, impurity, maxDepth, maxBins)
          for impurity in impurityList
          for maxDepth in maxDepthList
          for maxBins in maxBinsList]

训练评估：参数impurity=gini   maxDepth=10   maxBins=3
===>消耗时间=1.8239459991455078   结果AUC=0.6680894595738677
训练评估：参数impurity=gini   maxDepth=10   maxBins=5
===>消耗时间=1.4326863288879395   结果AUC=0.6270283617891914
训练评估：参数impurity=gini   maxDepth=10   maxBins=10
===>消耗时间=1.2209198474884033   结果AUC=0.6431558423233321
训练评估：参数impurity=gini   maxDepth=10   maxBins=15
===>消耗时间=1.099700689315796   结果AUC=0.6381851533537719
训练评估：参数impurity=gini   maxDepth=10   maxBins=100
===>消耗时间=1.400726079940796   结果AUC=0.6595911848839746
训练评估：参数impurity=gini   maxDepth=10   maxBins=200
===>消耗时间=1.6041264533996582   结果AUC=0.6563329784368306


### 4、网格搜索最佳参数组合

In [54]:
## 定义函数gridSearch网格搜索最佳参数组合

def gridSearch(trainData, validationData, impurityList, maxDepthList, maxBinsList ):
    metrics = [trainEvaluateModel(trainData, validationData, impurity, maxDepth, maxBins)
          for impurity in impurityList
          for maxDepth in maxDepthList
          for maxBins in maxBinsList]
    # 按照AUC从大到小排序，返回最大AUC的参数组合
    sorted_metics = sorted(metrics, key=lambda k:k[0], reverse=True)
    best_parameters = sorted_metics[0]
    print("最佳参数组合："+"impurity="+str( best_parameters[2]) + 
         ",  maxDepth="+str( best_parameters[3])+",  maxBins="+str( best_parameters[4])+"\n"+
         ",  结果AUC="+str( best_parameters[0]))
    return  best_parameters
## 参数组合
impurityList=["gini", "entropy"]
maxDepthList = [3,5,10,15,20,25]
maxBinsList = [3,5,10,15,100,200]

## 调用函数返回最佳参数组合
best_parameters = gridSearch(trainData, validationData, impurityList, maxDepthList, maxBinsList)

最佳参数组合：impurity=entropy,  maxDepth=10,  maxBins=200
,  结果AUC=0.6733680105699297


# 9  判断是否过拟合以及模型预测
### 1、判断是否过拟合

前面已经得到最佳参数组合impurity=entropy, maxDepth=10, maxBins=15及相应的AUC评估。使用该最佳参数组合作用于测试数据，是否会过拟合：

In [55]:
## 定义模型评估函数
def evaluateModel(model, validationData):
    predict = model.predict(validationData.map(lambda p:p.features))
    predict_real = predict.zip(validationData.map(lambda p: p.label))
    metrics = BinaryClassificationMetrics(predict_real)
    return metrics.areaUnderROC

## 使用最佳参数组合impurity=entropy,  maxDepth=10,  maxBins=15训练模型
best_model = DecisionTree.trainClassifier(trainData, numClasses=2,categoricalFeaturesInfo={}, 
                                         impurity="entropy", maxDepth=10,maxBins=15)
AUC1 = evaluateModel(best_model, trainData)
AUC2 = evaluateModel(best_model, testData)
print("training: AUC="+str(AUC1))
print("testing: AUC="+str(AUC2))

training: AUC=0.7736970524550343
testing: AUC=0.6274975225772675


该模型在测试集上的准确率比训练集上的准确率差别较大，可以判定发生了过拟合。

## 2、使用模型进行预测

如果不考虑过拟合，依然使用上面最佳参数组合的模型对test.tsv进行预测，返回预测结果

In [57]:
## 使用模型进行预测
def predictData(sc,model,categoriesMap):
    print("开始导入数据...")
    rawDataWithHeader = sc.textFile("data/test.tsv")
    ## 取第一项数据
    header = rawDataWithHeader.first()
    ## 剔除字段名（特征名）行，取数据行
    rawData = rawDataWithHeader.filter(lambda x:x!=header)
    ## 将双引号"替换为空字符（剔除双引号）
    rData = rawData.map(lambda x:x.replace("\"",""))
    ## 以制表符分割每一行
    lines = rData.map(lambda x: x.split("\t"))
    ## 预处理测试数据集
    testDataRDD=lines.map(lambda r: (r[0], process_features(r, categoriesMap, len(r))))
    DescDict={0:"暂时型(ephemeral)网页",
              1:"长久型(evergreen)网页"}
    ## 预测前5项数据
    for testData in testDataRDD.take(5):
        predictResult=model.predict(testData[1])
        print("网址："+str(testData[0])+"\n"+" ===>预测结果为: "+str(predictResult) + "说明: "+DescDict[predictResult]+"\n")

predictData(sc,best_model,categoriesMap)

开始导入数据...
网址：http://www.lynnskitchenadventures.com/2009/04/homemade-enchilada-sauce.html
 ===>预测结果为: 1.0说明: 长久型(evergreen)网页

网址：http://lolpics.se/18552-stun-grenade-ar
 ===>预测结果为: 0.0说明: 暂时型(ephemeral)网页

网址：http://www.xcelerationfitness.com/treadmills.html
 ===>预测结果为: 0.0说明: 暂时型(ephemeral)网页

网址：http://www.bloomberg.com/news/2012-02-06/syria-s-assad-deploys-tactics-of-father-to-crush-revolt-threatening-reign.html
 ===>预测结果为: 0.0说明: 暂时型(ephemeral)网页

网址：http://www.wired.com/gadgetlab/2011/12/stem-turns-lemons-and-limes-into-juicy-atomizers/
 ===>预测结果为: 1.0说明: 长久型(evergreen)网页



# 10 查看决策树分类规则

In [59]:
## 查看决策树分类规则
# best_model.toDebugString()

## 作业

上述模型发生了过拟合，请给出不会过拟合的模型，并给出预测测试结果和分类规则。